# Collect Result

Author: Bingcheng Jin

Date: 2024.1.1

In [22]:
import os
import glob

import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astropy.table import Table, vstack

In [23]:
band_labels = ['F115W','F150W','F200W','F277W','F356W','F410M','F444W']
nbands = len(band_labels)

In [24]:
sample_base = '../io/sample/'
img_dir_list = glob.glob(sample_base + '*/')
img_dir_list.sort()
img_dir_list

['../io/sample/nircam1/',
 '../io/sample/nircam2/',
 '../io/sample/nircam3/',
 '../io/sample/nircam6/']

In [25]:
tab_row = []
for sample_base_dir in img_dir_list:
    pointing = sample_base_dir.split('/')[-2]
    sample_dir_list = glob.glob(sample_base_dir + '*/')
    sample_dir_list.sort()
    for sample_dir in sample_dir_list:
        id = sample_dir.split('/')[-2] # plz consider naming convention
        xclist, yclist, maglist, relist, nlist, arlist, palist = [], [], [], [], [], [], []
        xcelist, ycelist, magelist, reelist, nelist, arelist, paelist = [], [], [], [], [], [], []
        for band in band_labels:
            output_path = sample_dir + f'output_{band}.fits'
            if not os.path.exists(output_path):
                print(f'{band} Output Not Exist in ',sample_dir)
                continue
            with fits.open(output_path) as hdul:
                tab_info = Table(hdul['FIT_INFO'].data)
                fit_sec = tab_info['FITSECT'][0].split(' ')[0]
                chi2nu = float(tab_info['CHI2NU'].data[0])
                row_band = Table(hdul['FINAL_BAND'].data)[0]
                xc, xc_e = row_band['COMP1_XC'], row_band['COMP1_XC_ERR']
                yc, yc_e = row_band['COMP1_YC'], row_band['COMP1_YC_ERR']
                mag, mag_e = row_band['COMP1_MAG'], row_band['COMP1_MAG_ERR']
                re, re_e = row_band['COMP1_Re'], row_band['COMP1_Re_ERR']
                n, n_e = row_band['COMP1_n'], row_band['COMP1_n_ERR']
                ar, ar_e = row_band['COMP1_AR'], row_band['COMP1_AR_ERR']
                pa, pa_e = row_band['COMP1_PA'], row_band['COMP1_PA_ERR']
            
            xclist.append(xc), xcelist.append(xc_e)
            yclist.append(yc), ycelist.append(yc_e)
            maglist.append(mag), magelist.append(mag_e)
            relist.append(re), reelist.append(re_e)
            nlist.append(n), nelist.append(n_e)
            arlist.append(ar), arelist.append(ar_e)
            palist.append(pa), paelist.append(pa_e)

        row = [id, pointing, chi2nu, fit_sec, xclist, xcelist, yclist, ycelist, maglist, magelist, relist, reelist, nlist, nelist, arlist, arelist, palist, paelist]
        tab_row.append(row)
tab_result = Table(rows=tab_row, names=['ID','NIRCamPointing', 'CHI2NU', 'FIT_SECTION', 'XC', 'XC_ERR', 'YC', 'YC_ERR', 'MAG', 'MAG_ERR', 'Re', 'Re_ERR', 'n', 'n_ERR', 'AR', 'AR_ERR', 'PA', 'PA_ERR'])

In [26]:
tab_result

ID,NIRCamPointing,CHI2NU,FIT_SECTION,XC,XC_ERR,YC,YC_ERR,MAG,MAG_ERR,Re,Re_ERR,n,n_ERR,AR,AR_ERR,PA,PA_ERR
str5,str7,float64,str13,float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7]
20901,nircam1,1.0296777486801147,"[1:166,1:166]",83.94934 .. 84.57736,0.049227897 .. 0.06054387,82.46254 .. 83.8065,0.09844307 .. 0.09068725,26.49936 .. 25.27471,0.029893339 .. 0.023447257,3.210626 .. 6.287482,0.11000902 .. 0.25861475,0.11140405 .. 1.8790381,0.080799736 .. 0.14261965,0.3376099 .. 0.33986408,0.017903147 .. 0.021563718,-19.692446 .. -4.918436,1.6996852 .. 1.4175128
21112,nircam1,1.081520676612854,"[1:152,1:152]",80.0 .. 76.8648,0.14894949 .. 0.010179053,76.834854 .. 77.81406,0.11522259 .. 0.012388358,26.709244 .. 23.347229,0.106652856 .. 0.0028228604,3.6239424 .. 3.8937805,0.6098574 .. 0.016718829,1.6061838 .. 1.0288035,0.40605474 .. 0.01879325,0.6758742 .. 0.6819737,0.088367805 .. 0.004977987,-56.033695 .. -20.131153,11.609171 .. 0.65755737
21844,nircam1,1.014214038848877,"[1:132,1:132]",67.47615 .. 67.84705,0.05066623 .. 0.02683698,66.8217 .. 67.40998,0.03551374 .. 0.021323757,24.934427 .. 24.108734,0.015860647 .. 0.0066139684,4.330886 .. 4.5541196,0.11200308 .. 0.053315174,0.9978136 .. 1.1794597,0.044752788 .. 0.034820307,0.6499154 .. 0.672004,0.013745732 .. 0.009209394,70.05633 .. 76.267586,1.960164 .. 1.189506
21879,nircam1,1.030426263809204,"[1:150,1:150]",79.0 .. 79.0,0.2817464 .. 0.15434842,76.11873 .. 76.28996,0.074597955 .. 0.06131481,25.503569 .. 25.222692,0.050090794 .. 0.022795228,12.677013 .. 9.870407,1.0013607 .. 0.3806904,1.5304585 .. 1.1852841,0.15296689 .. 0.09760649,0.15584795 .. 0.1850999,0.010167143 .. 0.010496685,86.566185 .. -89.70848,0.88684183 .. 0.7519423
21944,nircam1,1.0615006685256958,"[1:170,1:170]",86.40816 .. 86.39128,0.105640665 .. 0.04326789,86.637344 .. 86.392365,0.06591955 .. 0.029637558,27.184568 .. 23.870085,0.08939488 .. 0.10134357,0.7434691 .. 0.119967215,0.14518741 .. 0.06868107,5.3659716 .. 10.0,3.390544 .. 3.8796813,0.18907441 .. 0.5655661,0.12195655 .. 0.07610123,25.085958 .. -4.4183097,5.145902 .. 6.3891554
21984,nircam1,0.9855358004570007,"[1:110,1:110]",56.2688 .. 56.28083,0.054020964 .. 0.045921396,57.3707 .. 57.249084,0.11132402 .. 0.06526887,26.351555 .. 25.172955,0.03416704 .. 0.019376593,3.8791149 .. 4.422244,0.21004187 .. 0.122437194,0.8472594 .. 1.7397485,0.106457666 .. 0.18468505,0.2994422 .. 0.40158266,0.020519739 .. 0.023817116,14.999291 .. 11.14384,1.7179011 .. 1.6364778
22161,nircam1,0.9777387380599976,"[1:96,1:96]",49.8716 .. 49.434475,0.056150306 .. 0.05051838,50.774567 .. 50.61866,0.15877222 .. 0.10283436,25.984304 .. 25.369848,0.035325605 .. 0.01257755,5.758141 .. 6.4116125,0.3358147 .. 0.1866746,0.98366445 .. 1.0616632,0.126126 .. 0.11200004,0.1917192 .. 0.15591587,0.013771553 .. 0.021403473,18.834944 .. 21.514767,1.1023651 .. 0.8409778
22209,nircam1,1.0209424495697021,"[1:124,1:124]",62.93899 .. 63.529186,0.04559647 .. 0.04167563,63.047398 .. 63.21463,0.041173346 .. 0.04156149,24.886295 .. 24.514156,0.016629994 .. 0.011512207,5.588819 .. 7.1140146,0.14978726 .. 0.13831797,1.164875 .. 1.1903032,0.05257417 .. 0.0519233,0.27751672 .. 0.3782435,0.006148544 .. 0.008947097,47.641098 .. 44.860603,0.59281456 .. 0.6929803
22211,nircam1,1.0057966709136963,"[1:124,1:124]",63.074253 .. 62.56761,0.01912978 .. 0.010439872,64.65354 .. 64.0198,0.021643827 .. 0.013361664,24.435524 .. 23.423164,0.012244614 .. 0.0042321645,3.3791542 .. 4.8150234,0.066971086 .. 0.033524584,1.3300265 .. 1.4875594,0.043938212 .. 0.025230214,0.8138133 .. 0.5842401,0.0134928655 .. 0.005296131,-34.329407 .. -31.12366,2.998 .. 0.5193599


In [27]:
tab_result.write('../result/result_egs_single.ecsv', overwrite=True)

## Collect Multi-Component Results

In [28]:
mcomp_dir = '../multicomp/'

In [29]:
tab_row = []
for sample_base_dir in img_dir_list:
    pointing = sample_base_dir.split('/')[-2]
    sample_dir_list = glob.glob(sample_base_dir + '*/')
    sample_dir_list.sort()
    for sample_dir in sample_dir_list:
        id = sample_dir.split('/')[-2] # plz consider naming convention
        if os.path.exists(mcomp_dir + id):
            sample_dir = mcomp_dir + id + '/'
            isolate = 0
        else:
            isolate = 1
        xclist, yclist, maglist, relist, nlist, arlist, palist = [], [], [], [], [], [], []
        xcelist, ycelist, magelist, reelist, nelist, arelist, paelist = [], [], [], [], [], [], []
        for band in band_labels:
            output_path = sample_dir + f'output_{band}.fits'
            if not os.path.exists(output_path):
                print(f'{band} Output Not Exist in ',sample_dir)
                continue
            with fits.open(output_path) as hdul:
                tab_info = Table(hdul['FIT_INFO'].data)
                fit_sec = tab_info['FITSECT'][0].split(' ')[0]
                chi2nu = float(tab_info['CHI2NU'].data[0])
                row_band = Table(hdul['FINAL_BAND'].data)[0]
                xc, xc_e = row_band['COMP1_XC'], row_band['COMP1_XC_ERR']
                yc, yc_e = row_band['COMP1_YC'], row_band['COMP1_YC_ERR']
                mag, mag_e = row_band['COMP1_MAG'], row_band['COMP1_MAG_ERR']
                re, re_e = row_band['COMP1_Re'], row_band['COMP1_Re_ERR']
                n, n_e = row_band['COMP1_n'], row_band['COMP1_n_ERR']
                ar, ar_e = row_band['COMP1_AR'], row_band['COMP1_AR_ERR']
                pa, pa_e = row_band['COMP1_PA'], row_band['COMP1_PA_ERR']
            
            xclist.append(xc), xcelist.append(xc_e)
            yclist.append(yc), ycelist.append(yc_e)
            maglist.append(mag), magelist.append(mag_e)
            relist.append(re), reelist.append(re_e)
            nlist.append(n), nelist.append(n_e)
            arlist.append(ar), arelist.append(ar_e)
            palist.append(pa), paelist.append(pa_e)

        row = [id, pointing, chi2nu, fit_sec, xclist, xcelist, yclist, ycelist, maglist, magelist, relist, reelist, nlist, nelist, arlist, arelist, palist, paelist, isolate]
        tab_row.append(row)
tab_result = Table(rows=tab_row, names=['ID','NIRCamPointing', 'CHI2NU', 'FIT_SECTION', 'XC', 'XC_ERR', 'YC', 'YC_ERR', 'MAG', 'MAG_ERR', 'Re', 'Re_ERR', 'n', 'n_ERR', 'AR', 'AR_ERR', 'PA', 'PA_ERR', 'ISOLATE'])

In [30]:
tab_result.write('../result/result_egs_single_multicomp.ecsv', overwrite=True)

In [21]:
tab_result[tab_result['ISOLATE'] == 1]

ID,NIRCamPointing,CHI2NU,FIT_SECTION,XC,XC_ERR,YC,YC_ERR,MAG,MAG_ERR,Re,Re_ERR,n,n_ERR,AR,AR_ERR,PA,PA_ERR,ISOLATE
str5,str7,float64,str13,float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],float32[7],int64
21944,nircam1,1.0615006685256958,"[1:170,1:170]",86.40816 .. 86.39128,0.105640665 .. 0.04326789,86.637344 .. 86.392365,0.06591955 .. 0.029637558,26.735022 .. 23.946884,0.08939488 .. 0.10134357,0.7434691 .. 0.119967215,0.14518741 .. 0.06868107,5.3659716 .. 10.0,3.390544 .. 3.8796813,0.18907441 .. 0.5655661,0.12195655 .. 0.07610123,25.085958 .. -4.4183097,5.145902 .. 6.3891554,1
21984,nircam1,0.9855358004570007,"[1:110,1:110]",56.2688 .. 56.28083,0.054020964 .. 0.045921396,57.3707 .. 57.249084,0.11132402 .. 0.06526887,26.141985 .. 25.249754,0.03416704 .. 0.019376593,3.8791149 .. 4.422244,0.21004187 .. 0.122437194,0.8472594 .. 1.7397485,0.106457666 .. 0.18468505,0.2994422 .. 0.40158266,0.020519739 .. 0.023817116,14.999291 .. 11.14384,1.7179011 .. 1.6364778,1
22209,nircam1,1.0209424495697021,"[1:124,1:124]",62.93899 .. 63.529186,0.04559647 .. 0.04167563,63.047398 .. 63.21463,0.041173346 .. 0.04156149,24.795387 .. 24.71321,0.016629994 .. 0.011512207,5.588819 .. 7.1140146,0.14978726 .. 0.13831797,1.164875 .. 1.1903032,0.05257417 .. 0.0519233,0.27751672 .. 0.3782435,0.006148544 .. 0.008947097,47.641098 .. 44.860603,0.59281456 .. 0.6929803,1
22211,nircam1,1.0057966709136963,"[1:124,1:124]",63.074253 .. 62.56761,0.01912978 .. 0.010439872,64.65354 .. 64.0198,0.021643827 .. 0.013361664,24.344618 .. 23.622217,0.012244614 .. 0.0042321645,3.3791542 .. 4.8150234,0.066971086 .. 0.033524584,1.3300265 .. 1.4875594,0.043938212 .. 0.025230214,0.8138133 .. 0.5842401,0.0134928655 .. 0.005296131,-34.329407 .. -31.12366,2.998 .. 0.5193599,1
22267,nircam1,1.049236536026001,"[1:268,1:268]",134.29405 .. 135.61209,1.4862989 .. 0.0132351285,153.05779 .. 133.26517,1.8203282 .. 0.028191764,25.620815 .. 22.659195,0.11067127 .. 0.002513382,26.207058 .. 13.446016,1.8352734 .. 0.052409757,0.22266945 .. 0.86752164,0.08515107 .. 0.008356327,0.7182479 .. 0.32003996,0.057044294 .. 0.001204289,-40.42687 .. -24.604773,11.723806 .. 0.12080269,1
22572,nircam1,1.0938996076583862,"[1:202,1:202]",103.284546 .. 101.959175,0.05229481 .. 0.01868572,105.11693 .. 104.62296,0.02610057 .. 0.012014584,25.173119 .. 23.481808,0.025974348 .. 0.0048911143,4.9217534 .. 6.7933383,0.21148634 .. 0.0578212,1.6736215 .. 1.6289096,0.107147396 .. 0.024027672,0.32040223 .. 0.42996913,0.011372993 .. 0.0041624713,-65.43498 .. -65.59211,0.9305051 .. 0.3214461,1
22780,nircam1,0.9863994121551514,"[1:136,1:136]",70.16232 .. 70.3542,0.0794444 .. 0.050490763,69.86339 .. 70.55587,0.101935536 .. 0.06029754,25.31515 .. 24.722378,0.033054702 .. 0.016102316,6.6234174 .. 7.773602,0.34301025 .. 0.21813843,1.1561279 .. 1.5736976,0.084931396 .. 0.07882962,0.38053158 .. 0.3617725,0.014427024 .. 0.012311916,-37.374584 .. -38.35569,1.4861571 .. 0.95835084,1
23225,nircam1,0.9063494205474854,"[1:160,1:160]",81.11018 .. 80.79729,0.11084013 .. 0.032180697,81.165504 .. 81.29468,0.0876302 .. 0.028632391,24.61759 .. 23.649975,0.09724667 .. 0.018631866,20.481943 .. 13.228568,3.4618015 .. 0.43656537,3.7942178 .. 3.2248256,0.35831475 .. 0.08812898,0.66384757 .. 0.8169457,0.04010205 .. 0.012066709,-82.496376 .. -63.71548,4.720278 .. 2.3599572,1
23708,nircam1,1.0053939819335938,"[1:130,1:130]",68.2978 .. 67.824326,0.37373662 .. 0.15132731,66.02177 .. 66.70601,0.23907037 .. 0.12280491,25.977339 .. 25.884346,0.05217766 .. 0.020841291,7.737532 .. 6.154394,0.58422464 .. 0.26499328,0.6362162 .. 0.78302324,0.11145011 .. 0.09938048,0.60721457 .. 0.7302983,0.04033971 .. 0.033196922,62.115307 .. 69.38001,5.851737 .. 5.5097666,1
